# CIFAR10을 CNN으로 학습하기. 여러 학습 방법으로 비교

In [54]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

#(8) learning rate decay
from torch.optim import lr_scheduler

batch_size=64
learning_rate=0.05
num_epoch=50

# 1. CIFAR10 train, test dataset 가져오기 (163 MB)

#### MNIST: 11MB

In [55]:
# cifar_train=dset.CIFAR10("CIFAR10/", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
# cifar_test=dset.CIFAR10("CIFAR10/", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

# cifar_train=dset.CIFAR10("CIFAR10/", train=True, 
#                          transform=transforms.Compose([
#                              transforms.Scale(36),
#                              transforms.CenterCrop(32),
#                              transforms.RandomHorizontalFlip(),
#                              transforms.Lambda(lambda x: x.rotate(90)),
#                              transforms.ToTensor()
#                          ]))

# (4) Data Normalization
cifar_train=dset.CIFAR10("CIFAR10/", train=True, 
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                         ])
                        , target_transform=None, download=True)

cifar_test=dset.CIFAR10("CIFAR10/", train=False, 
                         transform=transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
                         ])
                        , target_transform=None, download=True)

Files already downloaded and verified
Files already downloaded and verified


# 2. 대략적인 데이터 형태

In [56]:
# print ("cifar_train 길이:", len(cifar_train))
# print ("cifar_test 길이:", len(cifar_test))

# # 데이터 하나 형태
# image, label=cifar_train.__getitem__(1)  # 1번째 데이터
# print ("image data 형태:", image.size())
# print ("label: ", label)

# # 그리기
# img=image.numpy()  # image 타입을 numpy로 변환(3, 32, 32)

# # (3,32,32) -> (32,32,3)
# r,g,b=img[0,:,:], img[1,:,:], img[2,:,:]
# #img=img.reshape(img.shape[1], img.shape[2], img.shape[0])
# img2=np.zeros((img.shape[1], img.shape[2], img.shape[0]))
# img2[:,:,0], img2[:,:,1], img2[:,:,2]=r,g,b

# plt.title("lavel: %d" %label)
# plt.imshow(img2, interpolation='bicubic')
# plt.show()

In [57]:
def ComputeAccr(dloader, imodel):
    correct=0
    total=0
    
    for j, [imgs, labels] in enumerate(dloader):  # batch_size 만큼
        img=Variable(imgs,volatile=True).cuda()  # x
        #label=Variable(labels)  # y
        label = Variable(labels).cuda()  # GPU 가속시
        # .cuda() : GPU 에 로드되기 위함. 만약 CPU로 설정되어 있다면 에러남
        
        
        output=imodel.forward(img)  # forward prop.
        _, output_index=torch.max(output, 1)
        
        total += label.size(0)
        correct += (output_index == label).sum().float()
    print("Accuracy of Test Data: {}".format(100*correct/total))

In [58]:
# === 3. 데이터 로드함수 ===
train_loader=torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

# === 4. 모델 선언 ===
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer=nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ReLU(),
            nn.Dropout2d(0,2),  # (2) drop out
            nn.BatchNorm2d(16),  # (6) Batch normalization
            nn.Conv2d(16,32,3,padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer=nn.Sequential(
            nn.Linear(64*8*8, 100),
            nn.ReLU(),
            nn.Dropout2d(0.2),
            nn.BatchNorm1d(100),
            nn.Linear(100,10)
        )
        
        # (3) weight initialization
        # for m in self.modules():
        #     if isinstance(m, nn.Conv2d):
        #         init.kaiming_normal(m.weight.data)  # RELU 일 때
        #         m.bias.data.fill_(0)
        #     if isinstance(m, nn.Linear):
        #         init.kaiming_normal(m.weight.data)
        #         m.bias.data.fill_(0)
    
    def forward(self, x):
        out=self.layer(x)
        out=out.view(batch_size,-1)
        out=self.fc_layer(out)
        
        return out
model=CNN().cuda()

In [59]:
# === 5. loss, optimizer ===
loss_func=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)  # (6) Adam optimizer

scheduler=lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)  # (7) learning rate decay

model.train()  
# === 6. 학습 ===
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x=Variable(image).cuda()
        y_=Variable(label).cuda()
        
        optimizer.zero_grad()
        output=model.forward(x)
        loss=loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
        if j%1000==0:
            print(j,loss)

0 tensor(2.3700, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.2120, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.0881, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8726, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(1.3772, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8793, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6987, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8523, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6935, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8147, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.8952, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6776, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.5263, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.4938, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.4948, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.3449, device='cuda:0', grad_fn=<NllLossBackward>)
0 tensor(0.6403, device=

In [60]:
model.eval()
ComputeAccr(test_loader, model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


Accuracy of Test Data: 76.65264892578125


In [62]:
netname='./mlp_weight.pkl'
torch.save(model, netname, )